<a href="https://colab.research.google.com/github/RareDiseaseData/rd_simulations/blob/main/regression_approach.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Libraries

In [1]:
import pandas as pd

In [2]:
df = pd.read_excel('/content/sample dataset.xlsx', sheet_name = 'Sheet1')
df

,ID,Author,Year,Title,Cell 1,Cell2,Age,Sex,OI Type,n,Contains std,type of comparaison,Method,Control vs patient vs aggregate,Measurement,% hydroxylysine
0,6,"Bleckmann, H.;Kresse, H.;Wollensak, J.;Buddeck...",1971,[Glycosaminoglycan and collagen analyses in os...,2,2,2,1,9,NaN,No,1,1.0,1,1,0.465116
1,6,"Bleckmann, H.;Kresse, H.;Wollensak, J.;Buddeck...",1971,[Glycosaminoglycan and collagen analyses in os...,2,2,2,1,10,NaN,No,1,1.0,2,1,0.395257
2,8,"Eastoe, J. E.;Martens, P.;Thomas, N. R.",1973,The amino-acid composition of human hard tissu...,2,3,1,1,2,NaN,No,1,2.0,1,1,0.219020
3,8,"Eastoe, J. E.;Martens, P.;Thomas, N. R.",1973,The amino-acid composition of human hard tissu...,2,5,5,3,10,NaN,No,1,2.0,2,1,0.111111
4,15,"Trelstad, R. L.;Rubin, D.;Gross, J.",1977,Osteogenesis imperfecta congenita: evidence fo...,1,1,1,2,2,NaN,No,2,5.0,1,1,0.218750
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
125,352,"Makareeva, Elena;Mirigian, Lynn S.;Mertz, Edwa...",2018,Substitutions for arginine at position 780 in ...,1,1,5,3,9,NaN,Yes,3,7.0,1,1,0.290000
126,352,"Makareeva, Elena;Mirigian, Lynn S.;Mertz, Edwa...",2018,Substitutions for arginine at position 780 in ...,1,1,5,3,9,NaN,Yes,3,7.0,1,1,0.250000
127,352,"Makareeva, Elena;Mirigian, Lynn S.;Mertz, Edwa...",2018,Substitutions for arginine at position 780 in ...,1,1,5,3,10,NaN,Yes,3,7.0,2,1,0.210000
128,377,"Syx, D.;Ishikawa, Y.;Gebauer, J.;Boudko, S. P....",2021,Aberrant binding of mutant HSP47 affects postt...,1,1,2,2,17,NaN,No,1,7.0,1,1,0.489200


Column selection

In [3]:
df.columns

Index(['ID', 'Author', 'Year', 'Title', 'Cell 1', 'Cell2', 'Age', 'Sex',
       'OI Type', 'n', 'Contains std', 'type of comparaison', 'Method',
       'Control vs patient vs aggregate', 'Measurement', '% hydroxylysine'],
      dtype='object')

In [4]:
df_sel = df[[
  'Cell2',
  'Age',
  'Sex',
  'OI Type', # disease_severity; one hot encoding
  'n',
  'Method', # one hot enconding
  '% hydroxylysine' # value
]]
df_sel

,Cell2,Age,Sex,OI Type,n,Method,% hydroxylysine
0,2,2,1,9,NaN,1.0,0.465116
1,2,2,1,10,NaN,1.0,0.395257
2,3,1,1,2,NaN,2.0,0.219020
3,5,5,3,10,NaN,2.0,0.111111
4,1,1,2,2,NaN,5.0,0.218750
...,...,...,...,...,...,...,...
125,1,5,3,9,NaN,7.0,0.290000
126,1,5,3,9,NaN,7.0,0.250000
127,1,5,3,10,NaN,7.0,0.210000
128,1,2,2,17,NaN,7.0,0.489200


In [5]:
df_sel.columns = ['cell', 'age', 'sex', 'ds', 'n', 'method', 'value']
df_sel

,cell,age,sex,ds,n,method,value
0,2,2,1,9,NaN,1.0,0.465116
1,2,2,1,10,NaN,1.0,0.395257
2,3,1,1,2,NaN,2.0,0.219020
3,5,5,3,10,NaN,2.0,0.111111
4,1,1,2,2,NaN,5.0,0.218750
...,...,...,...,...,...,...,...
125,1,5,3,9,NaN,7.0,0.290000
126,1,5,3,9,NaN,7.0,0.250000
127,1,5,3,10,NaN,7.0,0.210000
128,1,2,2,17,NaN,7.0,0.489200


In [6]:
df_sel[df_sel.n > 1]

,cell,age,sex,ds,n,method,value
29,1,2,3,1,3.0,5.0,0.129901
30,1,1,3,2,5.0,5.0,0.260990
31,1,5,3,3,12.0,5.0,0.140198
32,1,5,3,4,3.0,5.0,0.121188
33,1,1,3,10,2.0,5.0,0.189307
34,1,4,3,10,8.0,5.0,0.123168
35,5,2,3,1,3.0,3.0,0.134257
36,5,1,3,2,5.0,3.0,0.271683
37,5,5,3,3,12.0,3.0,0.114455
38,5,5,3,4,3.0,3.0,0.152079


In [7]:
import numpy as np

In [8]:
# Random std [0.008, 0.011]
np.random.uniform(low=0.008, high=0.011, size=(1))[0]

0.010878160226612324

In [9]:
df_sel['std_'] = 0
df_sel

<ipython-input-9-d7425242b68c>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_sel['std_'] = 0


,cell,age,sex,ds,n,method,value,std_
0,2,2,1,9,NaN,1.0,0.465116,0
1,2,2,1,10,NaN,1.0,0.395257,0
2,3,1,1,2,NaN,2.0,0.219020,0
3,5,5,3,10,NaN,2.0,0.111111,0
4,1,1,2,2,NaN,5.0,0.218750,0
...,...,...,...,...,...,...,...,...
125,1,5,3,9,NaN,7.0,0.290000,0
126,1,5,3,9,NaN,7.0,0.250000,0
127,1,5,3,10,NaN,7.0,0.210000,0
128,1,2,2,17,NaN,7.0,0.489200,0


In [10]:
for i, r in df_sel.iterrows(): # iterating rows
  # i: id
  # r: row values
  if r.n > 1:
    df_sel.loc[i, 'std_'] = np.random.uniform(low=0.008, high=0.011, size=(1))[0]

df_sel[df_sel.n > 1]

/usr/local/lib/python3.9/dist-packages/pandas/core/indexing.py:1817: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_column(loc, value, pi)


,cell,age,sex,ds,n,method,value,std_
29,1,2,3,1,3.0,5.0,0.129901,0.010078
30,1,1,3,2,5.0,5.0,0.260990,0.009218
31,1,5,3,3,12.0,5.0,0.140198,0.008334
32,1,5,3,4,3.0,5.0,0.121188,0.008802
33,1,1,3,10,2.0,5.0,0.189307,0.008633
34,1,4,3,10,8.0,5.0,0.123168,0.008833
35,5,2,3,1,3.0,3.0,0.134257,0.008122
36,5,1,3,2,5.0,3.0,0.271683,0.009903
37,5,5,3,3,12.0,3.0,0.114455,0.009271
38,5,5,3,4,3.0,3.0,0.152079,0.010366


# Preprocessing

In [11]:
df_sel.isna().sum()

cell        0
age         0
sex         0
ds          0
n         117
method      6
value       0
std_        0
dtype: int64

In [12]:
# Filling n = NA with 1
df_sel.loc[df_sel.n.isna(), 'n'] = 1
df_sel.isna().sum()

/usr/local/lib/python3.9/dist-packages/pandas/core/indexing.py:1817: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_column(loc, value, pi)


cell      0
age       0
sex       0
ds        0
n         0
method    6
value     0
std_      0
dtype: int64

In [13]:
# Removing Maroteaux from method
df_sel = df_sel.dropna() # equivalent df_sel.dropna(inplace = True)
df_sel.isna().sum()

cell      0
age       0
sex       0
ds        0
n         0
method    0
value     0
std_      0
dtype: int64

In [14]:
# Convert disease_severity and method values into integers
df_sel.ds = df_sel.ds.astype(int)
df_sel.method = df_sel.method.astype(int)

/usr/local/lib/python3.9/dist-packages/pandas/core/generic.py:5516: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[name] = value


In [15]:
# One Hot Encoding
df_ds_dummies = pd.get_dummies(df_sel.ds, prefix='ds')
df_method_dummies = pd.get_dummies(df_sel.method, prefix='method')

In [17]:
df_sel.join(df_ds_dummies).join(df_method_dummies)

,cell,age,sex,ds,n,method,value,std_,ds_1,ds_2,...,ds_11,ds_13,ds_16,ds_17,method_1,method_2,method_3,method_5,method_6,method_7
0,2,2,1,9,1.0,1,0.465116,0.0,0,0,...,0,0,0,0,1,0,0,0,0,0
1,2,2,1,10,1.0,1,0.395257,0.0,0,0,...,0,0,0,0,1,0,0,0,0,0
2,3,1,1,2,1.0,2,0.219020,0.0,0,1,...,0,0,0,0,0,1,0,0,0,0
3,5,5,3,10,1.0,2,0.111111,0.0,0,0,...,0,0,0,0,0,1,0,0,0,0
4,1,1,2,2,1.0,5,0.218750,0.0,0,1,...,0,0,0,0,0,0,0,1,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
125,1,5,3,9,1.0,7,0.290000,0.0,0,0,...,0,0,0,0,0,0,0,0,0,1
126,1,5,3,9,1.0,7,0.250000,0.0,0,0,...,0,0,0,0,0,0,0,0,0,1
127,1,5,3,10,1.0,7,0.210000,0.0,0,0,...,0,0,0,0,0,0,0,0,0,1
128,1,2,2,17,1.0,7,0.489200,0.0,0,0,...,0,0,0,1,0,0,0,0,0,1


# Data selection

In [43]:
df_train = df_sel[df_sel.std_ != 0]
df_train

,cell,age,sex,disease_severity,n,method,value,std_
29,1,2,3,1,3.0,5.0,0.129901,0.008232
30,1,1,3,2,5.0,5.0,0.260990,0.010024
31,1,5,3,3,12.0,5.0,0.140198,0.009387
32,1,5,3,4,3.0,5.0,0.121188,0.008290
33,1,1,3,10,2.0,5.0,0.189307,0.010765
34,1,4,3,10,8.0,5.0,0.123168,0.009716
35,5,2,3,1,3.0,3.0,0.134257,0.010093
36,5,1,3,2,5.0,3.0,0.271683,0.008441
37,5,5,3,3,12.0,3.0,0.114455,0.008039
38,5,5,3,4,3.0,3.0,0.152079,0.008120


In [44]:
X = df_train[[
  'cell',
  'age',
  'sex',
  'ds',
  'n',
  'method',
  'value'
]]
X

,cell,age,sex,disease_severity,n,method,value
29,1,2,3,1,3.0,5.0,0.129901
30,1,1,3,2,5.0,5.0,0.260990
31,1,5,3,3,12.0,5.0,0.140198
32,1,5,3,4,3.0,5.0,0.121188
33,1,1,3,10,2.0,5.0,0.189307
34,1,4,3,10,8.0,5.0,0.123168
35,5,2,3,1,3.0,3.0,0.134257
36,5,1,3,2,5.0,3.0,0.271683
37,5,5,3,3,12.0,3.0,0.114455
38,5,5,3,4,3.0,3.0,0.152079


In [45]:
Y = df_train[['std_']]
Y

,std_
29,0.008232
30,0.010024
31,0.009387
32,0.008290
33,0.010765
34,0.009716
35,0.010093
36,0.008441
37,0.008039
38,0.008120


# Linear regression

In [46]:
from sklearn import linear_model

In [47]:
reg = linear_model.LinearRegression()

In [48]:
reg.fit(X = X, y = Y)

LinearRegression()

In [49]:
reg.coef_

array([[ 5.95473152e-05, -4.28878934e-04, -1.43385737e-17,
         1.63099163e-04,  7.51912881e-05,  1.68607207e-04,
        -4.21859858e-03]])

In [50]:
reg.intercept_

array([0.00923588])

In [51]:
df_test = df_sel[df_sel.std_ == 0]

In [53]:
new_X = df_test[[
  'cell',
  'age',
  'sex',
  'ds',
  'n',
  'method',
  'value'
]]
new_X

,cell,age,sex,disease_severity,n,method,value
0,2,2,1,9,1.0,1.0,0.465116
1,2,2,1,10,1.0,1.0,0.395257
2,3,1,1,2,1.0,2.0,0.219020
3,5,5,3,10,1.0,2.0,0.111111
4,1,1,2,2,1.0,5.0,0.218750
...,...,...,...,...,...,...,...
125,1,5,3,9,1.0,7.0,0.290000
126,1,5,3,9,1.0,7.0,0.250000
127,1,5,3,10,1.0,7.0,0.210000
128,1,2,2,17,1.0,7.0,0.489200


In [54]:
reg.predict(new_X)

array([[0.00824677],
       [0.00870457],
       [0.00880029],
       [0.00896388],
       [0.00918815],
       [0.0082664 ],
       [0.00840394],
       [0.01059926],
       [0.01045768],
       [0.01043063],
       [0.0088707 ],
       [0.00870637],
       [0.01067329],
       [0.01067032],
       [0.0092756 ],
       [0.00908048],
       [0.00941988],
       [0.00871329],
       [0.0089213 ],
       [0.00879086],
       [0.01049578],
       [0.00842353],
       [0.01036112],
       [0.0106986 ],
       [0.01052986],
       [0.00799376],
       [0.00868853],
       [0.00907086],
       [0.00911119],
       [0.00859227],
       [0.00997442],
       [0.00997442],
       [0.00763874],
       [0.00974506],
       [0.00889112],
       [0.00971083],
       [0.00973614],
       [0.00964231],
       [0.00651963],
       [0.00658033],
       [0.00687168],
       [0.00692024],
       [0.0069688 ],
       [0.00712257],
       [0.00878672],
       [0.0087422 ],
       [0.0085075 ],
       [0.008

- [ ] Separate healty from disease patients
- [ ] Normalize data; and after the prediction, denormalized it
- [ ] Simulate rows n > 1 into singletons
- [ ] Compare models based on the error metrics (or accuracy)
- [ ] Train-Test data analysis